In [60]:
from stock_util.nadex import Nadex
from datetime import date, datetime,timedelta
import pandas as pd
import numpy as np
import time
import math
import re



In [61]:
import json

from oandapyV20 import API
from oandapyV20.exceptions import V20Error
import oandapyV20.endpoints.instruments as instruments
from oandapyV20.definitions.instruments import CandlestickGranularity
#from exampleauth import exampleAuth

import json

nadex=Nadex()


Finished getting login


In [62]:
def exampleAuth():
    accountID, token = None, None
    with open("account.txt") as I:
        accountID = I.read().strip()
    with open("token.txt") as I:
        token = I.read().strip()
    return accountID, token

In [63]:
def get_quote_timeframe(minutesago=1):
    accountID, access_token = exampleAuth()
    params = {}
    params.update({"granularity": "M1"})
    params.update({"count": minutesago})
    params.update({"price": "BA"})
    r=instruments.InstrumentsCandles(instrument="EUR_USD",params=params)
    result=API(access_token).request(r)

    print("{}".format(json.dumps(result)))


In [64]:
get_quote_timeframe(2)

{"instrument": "EUR_USD", "granularity": "M1", "candles": [{"complete": true, "volume": 11, "time": "2019-06-28T20:58:00.000000000Z", "bid": {"o": "1.13691", "h": "1.13691", "l": "1.13679", "c": "1.13680"}, "ask": {"o": "1.13714", "h": "1.13717", "l": "1.13707", "c": "1.13717"}}, {"complete": true, "volume": 12, "time": "2019-06-28T20:59:00.000000000Z", "bid": {"o": "1.13662", "h": "1.13671", "l": "1.13646", "c": "1.13671"}, "ask": {"o": "1.13744", "h": "1.13771", "l": "1.13744", "c": "1.13771"}}]}


In [65]:
def login():
    try:
        #nadex=Nadex()
        nadex.login()
    except Exception as e:
        print(e)

In [66]:
def get_contracts():
    df = np.empty([3])
    
    p=re.compile(r'EUR/USD >(?P<price>\d+\.\d+)\s\((?P<end_hour>\d+)(?P<am_pm>\w+)\)')

    try:
        contracts=nadex.get_contracts()
    except Exception as e:
        print(e)

    now=time.mktime(datetime.now().timetuple())

    for contract in contracts:
        m=p.search(contract)

        if not m:
            continue

        price=m.group('price')
        end_hour=m.group('end_hour')
        am_pm=m.group('am_pm')

        if(am_pm == "PM"):
            hour24=int(end_hour)+12

        if(am_pm == "AM" and end_hour=="12"):
            hour24="00"
            end=datetime.now()+timedelta(hours=5) # append to next day. Assumes no options > 5 hours
        else:
            hour24=end_hour
            end=datetime.now()

        end=end.replace(hour=int(hour24),second=0,minute=0)
        #end=end.replace(minute=0)
        #end=end.replace(second=0)
        #print (end)
        end=time.mktime(end.timetuple())

        minutes_left=math.floor((end-now) / 60)
        
        print("Price: ",price," End:",end_hour," AM/PM:",am_pm, " 24h:",hour24, "Minutes Left: ",minutes_left)
        df.append([price,hour24,minutes_left])
    
    return pd.Dataframe(data=df)

In [67]:
login()
df=get_contracts()
print(df)

Found results iframe
Message: 

Couldn't get contracts elements


AttributeError: module 'pandas' has no attribute 'Dataframe'

In [ ]:
nadex.driver.quit()